In [1]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata
import itertools
from tqdm import tqdm

tqdm.pandas()

In [2]:
final_df = pd.read_feather("data/opioid_concept_map.feather")
full_bio_cohort = pd.read_feather('data/opioid_cohort_details.feather')

In [5]:
final_df

,person_id,10330-9,10333-3,10373-9,10378-8,10380-4,10621-1,10835-7,10839-9,10851-4,...,DOID:997,DOID:9970,DOID:9971,DOID:9972,DOID:9975,DOID:9976,DOID:9977,DOID:998,DOID:999,DOID:9993
0,304,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1516,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4565,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5127,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7451,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17794,10446618,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17795,10458182,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
17796,10458349,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
17797,10509936,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
full_bio_cohort

,person_id,first_presc_date,last_presc_date,n_visits,most_frequent_prescription,first_diagnosis_date,most_frequent_condition,dependent,gender_source_value,age,ethnicity_source_value,cutoff_date
0,1190427,2019-04-05,2023-02-09,26,40232756,2019-12-07,438120,True,Male,62,Hispanic or Latino,2019-10-05
1,1294727,1994-11-14,1995-08-14,3,1718699,2001-05-26,434016,True,Male,70,Not Hispanic or Latino,1995-05-14
2,3085465,2012-05-04,2016-04-14,240,40232756,2016-03-29,438120,True,Male,65,Not Hispanic or Latino,2012-11-04
3,1687475,2021-03-09,2023-02-21,23,40232756,2023-02-19,438120,True,Female,42,Not Hispanic or Latino,2021-09-09
4,530989,2010-10-17,2015-06-05,25,40162494,2011-10-29,438120,True,Female,91,Not Hispanic or Latino,2011-04-17
...,...,...,...,...,...,...,...,...,...,...,...,...
17819,10184203,2012-03-28,2013-03-20,3,40162515,None,0,False,Female,91,Not Hispanic or Latino,2012-09-28
17820,10224355,2015-04-07,2022-06-10,5,40162515,None,0,False,Male,62,Not Hispanic or Latino,2015-10-07
17821,9635335,2010-10-31,2023-01-15,137,40162511,None,0,False,Female,64,Not Hispanic or Latino,2011-04-30
17822,9639145,2018-06-23,2019-12-15,105,40232700,None,0,False,Female,74,Not Hispanic or Latino,2018-12-23


In [4]:
spoke = np.load('../../psev_repo/PSEV_matrix')
sep = np.load('../../psev_repo/PSEV_SEP_map')
spoke_node = np.load('../../psev_repo/PSEV_SPOKE_node_map')

spoke = pd.DataFrame(spoke, columns=spoke_node)
spoke.index = sep
spoke.index = spoke.index.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
spoke.columns = spoke.columns.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

In [5]:
spoke

,1,10,100,1000,10000,100008586,10001,10002,10003,100037417,...,X6RAB3,X6RAL5,X6RAN8,X6RAY8,X6RB12,X6REB3,X6REH9,X6RGC9,X6RLR1,X6RLX0
D010149,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
C0030201,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
C0007117,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003,...,0.000003,0.000004,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003
DOID:2513,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003,...,0.000003,0.000004,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003
C0022408,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000002,0.000002,0.000002,0.000003,...,0.000002,0.000002,0.000003,0.000002,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DB09297,0.000003,0.000003,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,...,0.000002,0.000002,0.000003,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
DB01176,0.000003,0.000002,0.000003,0.000003,0.000003,0.000003,0.000003,0.000004,0.000003,0.000003,...,0.000003,0.000003,0.000003,0.000003,0.000003,0.000004,0.000004,0.000003,0.000003,0.000003
DB00995,0.000002,0.000003,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
DB00726,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002


In [6]:
final_df

,person_id,C0000731,C0000735,C0000737,C0000768,C0000786,C0000814,C0000833,C0001361,C0001511,...,DOID:9969,DOID:997,DOID:9970,DOID:9971,DOID:9972,DOID:9975,DOID:9977,DOID:9988,DOID:9993,DOID:9997
0,1519,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4035,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4054,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4059,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4502,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6059,5735169,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6060,5735687,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6061,5736628,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6062,5738402,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
def generate_embedding(patient_concepts):
    #Filters out things that aren't in SPOKE
    patient_concepts = [item for item in patient_concepts if item in spoke.index]
    ind_psev = spoke.loc[patient_concepts].sum(axis=0)
    return ind_psev

In [8]:
num_patients = final_df.shape[0]

patient_embeddings = pd.DataFrame(index=range(num_patients), columns=list(spoke.columns))

#Goes through each record
for i in tqdm(range(0, final_df.shape[0])):
    #Grabs the non-zero concepts for that individual
    patient_concepts = final_df.columns[final_df.iloc[i] != 0].tolist()
    psev = generate_embedding(patient_concepts)
    patient_embeddings.loc[i] = psev

    #perform rankdata here so it remains a series, easier mapping as a row
    patient_embeddings.loc[i] = rankdata(patient_embeddings.loc[i])


patient_embeddings['person_id'] = final_df['person_id']

100%|██████████| 6064/6064 [1:55:28<00:00,  1.14s/it]  


In [9]:
patient_embeddings.to_feather("data/addiction_embeddings.feather")

In [2]:
embeddings = pd.read_feather("data/addiction_embeddings.feather")
embeddings

,1,10,100,1000,10000,100008586,10001,10002,10003,100037417,...,X6RAL5,X6RAN8,X6RAY8,X6RB12,X6REB3,X6REH9,X6RGC9,X6RLR1,X6RLX0,person_id
0,37400.0,255424.0,86562.0,54695.0,45865.0,71742.0,38549.0,17829.0,66042.0,63965.0,...,25071.0,101268.0,39291.5,70985.5,22922.0,4819.5,37013.0,28605.0,99823.0,1519
1,66518.0,170505.0,27722.0,38341.0,43020.0,69426.0,18287.0,27003.0,41278.0,54372.0,...,35105.0,69521.0,68115.5,82825.5,38870.5,19277.5,45794.0,35374.0,47802.5,4035
2,47818.0,88312.0,15192.0,45469.0,49076.0,71960.0,53371.0,14447.0,28243.0,68256.5,...,36173.0,69082.0,52937.5,70857.5,16557.5,23262.5,27747.5,25524.0,33105.5,4054
3,62250.0,128014.0,54179.0,44200.0,39926.0,68610.0,22870.0,24560.0,34557.0,57998.0,...,33969.0,124935.0,27172.0,63508.5,35297.5,17365.5,39864.0,32048.0,46073.0,4059
4,59808.0,365961.0,12737.0,39498.0,50086.0,86861.0,20291.0,33774.0,92190.0,56501.0,...,36165.0,85155.0,74806.5,218958.5,38084.5,14359.5,42711.0,38191.0,31696.0,4502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6059,63037.0,164422.0,25753.0,45288.0,38534.0,70532.5,34984.0,25153.0,42295.0,50152.0,...,39854.0,79754.0,20533.5,54038.0,34386.5,15868.5,41569.5,32954.0,66327.0,5735169
6060,59664.0,276477.0,23942.0,36237.0,54473.0,88924.0,3611.0,27933.0,356176.0,45767.0,...,32698.0,74535.0,322662.5,184945.5,32599.5,20098.5,56902.0,45087.0,44275.0,5735687
6061,59295.0,154196.0,56464.0,35850.5,40431.0,71191.0,21717.0,25327.0,28241.0,83477.0,...,38977.0,70847.0,8051.5,66362.5,39161.5,27129.5,43839.0,36979.5,34786.0,5736628
6062,62008.0,148613.0,28895.0,40338.0,46923.0,69489.0,16517.0,23323.0,36599.0,69050.0,...,37179.0,74825.0,20893.5,64931.5,34032.5,24224.0,45758.0,34392.0,40966.0,5738402


## Do Node type specific rank and then only select features with >70% variance